# Weekly assignment 8: Improving the discretized solution

In this exercise you will explore several alternatives in implementing up the
continuous choice solution for the cake eating problem.

## Task 1. Base solution and accuracy measure

Copy the version of the cake eating code with continuous choices
from the lecture and produce the convergence plots.

Import the check_analytic() fucntion from cake_eating.py module and make sure it works
as expected.  Assume the cake_eating.py module is in the same directory as this notebook.

In [ ]:
# write your code here
# come up with a test of your own

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
from scipy.optimize import minimize_scalar
%matplotlib inline

from cake_eating import check_analytic

class cake_continuous():
    '''Implementation of the cake eating problem with continuous choices.'''
    def __init__(self,beta=.9, Wbar=10, ngrid=50, interpolation='linear',maxiter_bellman=100,tol_bellman=1e-8):
        self.beta = beta    # Discount factor
        self.Wbar = Wbar    # Upper bound on cake size
        self.ngrid = ngrid  # Number of grid points for the size of cake
        self.epsilon = np.finfo(float).eps # smallest positive float number
        self.grid_state = np.linspace(self.epsilon,Wbar,ngrid) # grid for state space
        self.interpolation = interpolation # interpolation type for Bellman equation
        self.maxiter_bellman = maxiter_bellman # maximum iterations in Bellman solver
        self.tol_bellman = tol_bellman # tolerance in Bellman solver
    def bellman(self,V0):
        #Bellman operator, V0 is one-dim vector of values on grid
        def maximand(c,M,V0):
            '''Maximand of the Bellman equation'''
            # interpolation kind
            if self.interpolation=='linear':
                interfunc = interpolate.interp1d(self.grid_state,V0,kind='slinear',fill_value="extrapolate")
            elif self.interpolation=='quadratic':
                interfunc = interpolate.interp1d(self.grid_state,V0,kind='quadratic',fill_value="extrapolate")
            elif self.interpolation=='cubic':
                interfunc = interpolate.interp1d(self.grid_state,V0,kind='cubic',fill_value="extrapolate")
            elif self.interpolation=='polynomial':
                p = np.polynomial.polynomial.polyfit(self.grid_state,V0,self.ngrid-1)
                interfunc = lambda x: np.polynomial.polynomial.polyval(x,p)
            else:
                print('Unknown interpolation type')
                return None
            Vnext = interfunc(M-c) # next period value at the size of cake in the next period
            V1 = np.log(c) + self.beta*Vnext
            return -V1 # negative because of minimization
        def findC(M,maximand=None,V0=None):
            '''Solves for optimal consumption for given cake size M and value function VF'''
            opt = {'maxiter':self.maxiter_bellman, 'xatol':self.tol_bellman}
            res = minimize_scalar(maximand,args=(M,V0),method='Bounded',bounds=[self.epsilon,M],options=opt)
            if res.success:
                return res.x # if converged successfully
            else:
                return M/2 # return some visibly wrong value
        # loop over states
        c1=np.empty(self.ngrid,dtype='float')
        for i in range(self.ngrid):
            c1[i] = findC(self.grid_state[i],maximand,V0)
        V1 = - maximand(c1,self.grid_state,V0) # don't forget the negation!
        return V1, c1
    def solve(self, maxiter=1000, tol=1e-4, callback=None):
        '''Solves the model using successive approximations'''
        V0=np.log(self.grid_state) # on first iteration assume consuming everything
        for iter in range(maxiter):
            V1,c1=self.bellman(V0)
            if callback: callback(iter,self.grid_state,V1,c1) # callback for making plots
            if np.all(abs(V1-V0) < tol):
                break
            V0=V1
        else:  # when i went up to maxiter
            print('No convergence: maximum number of iterations achieved!')
        return V1,c1
    def solve_plot(self, maxiter=1000, tol=1e-4):
        '''Illustrate solution'''
        fig1, (ax1,ax2) = plt.subplots(1,2,figsize=(14,8))
        ax1.grid(b=True, which='both', color='0.65', linestyle='-')
        ax2.grid(b=True, which='both', color='0.65', linestyle='-')
        ax1.set_title('Value function convergence with VFI')
        ax2.set_title('Policy function convergence with VFI')
        ax1.set_xlabel('Cake size, W')
        ax2.set_xlabel('Cake size, W')
        ax1.set_ylabel('Value function')
        ax2.set_ylabel('Policy function')
        print('Iterations:',end=' ')
        def callback(iter,grid,v,c):
            print(iter,end=' ') # print iteration number
            ax1.plot(grid[1:],v[1:],color='k',alpha=0.25)
            ax2.plot(grid,c,color='k',alpha=0.25)
        V,c = self.solve(maxiter=maxiter,tol=tol,callback=callback)
        # add solutions
        ax1.plot(self.grid_state[1:],V[1:],color='r',linewidth=2.5)
        ax2.plot(self.grid_state,c,color='r',linewidth=2.5)
        plt.show()
        return V,c

In [ ]:
m = cake_continuous (beta=0.92,Wbar=10,ngrid=25,tol_bellman=1e-8)
V,c = m.solve_plot()
check_analytic(m,V=V,policy=c,title='Solution with continuous choices')

## Task 2. Improving interpolation method

We could utilize more advanced interpolation schemes for the value function itself.

Replace linear interpolation of the value function with quadratic and cubic splines, and approximating
polynomials.

Compare the accuracy of the new two versions to the original solution and the solution
with the improvement from task 3.

What is the most accurate solution algorithm?

In [ ]:
# write your code here

In [ ]:
for knd in 'linear','quadratic','cubic','polynomial':
    m.interpolation=knd
    V,c = m.solve(callback=lambda *arg: print(arg[0],end=' '))
    ac=check_analytic(m,V=V,policy=c)
    print('Accuracy with '+knd+' interpolation is',ac)

## Task 3. Convergence to true solution

Make a plot of the accuracy measure as function of number of grid points in the state space
for each of the four interpolation schemes.

Comparing the results in tasks 2 and 3, what is the best way to improve the accuracy of the solution?

In [ ]:
# write your code here

In [ ]:
fig1, ax1 = plt.subplots(figsize=(14,8))
ax1.set_xlabel('Number of grid points')
ax1.set_ylabel('Accuracy')
grids = np.arange(10,100,10,dtype='int')
for knd in 'linear','quadratic','cubic','polynomial':
    line = np.empty(grids.size)
    for i in range(grids.size):
        K = grids[i]
        m = cake_continuous(beta=0.92,Wbar=10,ngrid=grids[i])
        m.interpolation=knd
        V,c = m.solve(callback=lambda *arg: print(arg[0],end=' '))
        line[i]=check_analytic(m,V=V,policy=c,plot=False)
        print('.',end='')
    ax1.plot(grids,line,label=knd+' interpolation')
    print('|',end='')
ax1.legend()
plt.show()